## 🧮 Project Funding Optimization — Case Setup

**Date:** July 1, 2023  

Director of Special Projects **Rakesh Parameshwar** has a planned **$20.5 million project** that will require the following expected cash flows between **2024 and 2027**:

| Date | Cash Requirement ($mm) |
|:--|--:|
| 01-Jan-24 | 7.50 |
| 01-Jul-24 | 4.50 |
| 01-Jan-25 | 1.00 |
| 01-Jul-25 | 1.00 |
| 01-Jan-26 | 1.00 |
| 01-Jul-26 | 1.00 |
| 01-Jan-27 | 1.00 |
| 01-Jul-27 | 3.50 |

Rakesh asks **Christine Reyling**, Director of Financial Planning, to ensure that funding is available to meet these requirements.

Christine is considering purchasing a portfolio of **bonds**, with cash flows arranged to coincide with the project’s needs.  
The following bonds are available and can be purchased in any (even fractional) quantity:

| Maturity | Coupon | Current Price | Semiannual Coupon (per $1 face) |
|:--|--:|--:|--:|
| 01-Jan-24 | 7.00% | 1.00 | 0.0350 |
| 01-Jul-24 | 7.50% | 1.03 | 0.0375 |
| 01-Jan-25 | 6.75% | 1.02 | 0.0338 |
| 01-Jul-25 | 0.00% | 0.81 | 0.0000 |
| 01-Jan-26 | 10.00% | 1.16 | 0.0500 |
| 01-Jul-26 | 9.00% | 1.15 | 0.0450 |
| 01-Jan-27 | 10.25% | 1.23 | 0.0513 |
| 01-Jul-27 | 10.00% | 1.25 | 0.0500 |

Each bond pays **0.5 × (coupon rate)** every six months and the **face value at maturity**.

Example:  
- The 7.50% bond maturing on 01-Jul-24 pays  
  - \$0.0375 on 01-Jan-24  
  - \$1.0375 on 01-Jul-24  

---

### **Task**

Determine the **minimum-cost portfolio** of these bonds that will meet all project cash flow requirements exactly.  

- Bonds can be purchased fractionally.  
- Any interim excess cash can be reinvested at **4% annual (2% semiannual)**.  
- Ignore discounting — focus on matching cash flows.  

**Question:**  
> What is the *minimum-cost bond portfolio* (i.e., face values to buy of each bond) that satisfies the cash flow requirements for the project?



In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [5]:
#Data

dates = [
    "2024-01-01","2024-07-01","2025-01-01","2025-07-01",
    "2026-01-01","2026-07-01","2027-01-01","2027-07-01"
]
needs = np.array([7.50, 4.50, 1.00, 1.00, 1.00, 1.00, 1.00, 3.50])  # $mm

# Bonds (ordered by maturity date)

bond_data = pd.DataFrame({
    "Maturity": [
        "2024-01-01","2024-07-01","2025-01-01","2025-07-01",
        "2026-01-01","2026-07-01","2027-01-01","2027-07-01"
    ],
    "Coupon":  [0.0700, 0.0750, 0.0675, 0.0000, 0.1000, 0.0900, 0.1025, 0.1000],
    "Price":   [1.00,   1.03,   1.02,   0.81,   1.16,   1.15,   1.23,   1.25]
})


#Semi-annual coupon

bond_data["C_semi"] = bond_data["Coupon"] / 2

# Map maturities to timeline index (1..8)
maturity_to_idx = {d: i for i, d in enumerate(dates, start=1)}
m_idx = bond_data["Maturity"].map(maturity_to_idx).to_numpy()  # shape (8,)

n_dates = len(dates)
n_bonds = len(bond_data)

#Cash flow matrix

CF = np.zeros((n_dates, n_bonds))
for j in range(n_bonds):
    m = m_idx[j]
    CF[:m, j] += bond_data.loc[j, "C_semi"]
    CF[m-1, j] += 1.0  # principal at maturity (mth period)

# -----------------------------
# LP variables:
#   x_j  >= 0: face amount of bond j to buy (j=1..n_bonds)
#   s_t  >= 0: surplus after date t, carried to next period at 2% per half-year (t=1..n_dates)
# Dynamics:
#   s_1 = CF_1 x - need_1
#   s_t = 1.02 * s_{t-1} + CF_t x - need_t   for t=2..T
# Objective: minimize sum_j Price_j * x_j
# -----------------------------
grow = 1.02  # 2% per half-year

# Build equality constraints A_eq * var = b_eq
# var = [x_1..x_n, s_1..s_T]  -> length = n_bonds + n_dates
n_vars = n_bonds + n_dates
A_eq = np.zeros((n_dates, n_vars))
b_eq = np.zeros(n_dates)

# t = 1
A_eq[0, :n_bonds] = CF[0, :]
A_eq[0, n_bonds + 0] = -1.0  # -s_1
b_eq[0] = needs[0]

# t = 2..T
for t in range(1, n_dates):
    A_eq[t, :n_bonds] = CF[t, :]
    # + CF_t x - need_t - s_t + 1.02 s_{t-1} = 0
    A_eq[t, n_bonds + t] = -1.0             # -s_t
    A_eq[t, n_bonds + (t-1)] = grow         # +1.02*s_{t-1}
    b_eq[t] = needs[t]

# Bounds: x_j >= 0, s_t >= 0
bounds = [(0, None)] * n_vars

# Objective: minimize upfront cost
c = np.zeros(n_vars)
c[:n_bonds] = bond_data["Price"].to_numpy()

# Solve LP
res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

if not res.success:
    raise RuntimeError(f"LP failed: {res.message}")

x = res.x[:n_bonds]
s = res.x[n_bonds:]

# -----------------------------
# Reporting
# -----------------------------
portfolio = bond_data[["Maturity","Coupon","Price"]].copy()
portfolio["FaceValue_mm"] = x
portfolio["Cost_mm"] = portfolio["Price"] * portfolio["FaceValue_mm"]

total_cost = portfolio["Cost_mm"].sum()

# Reconstruct flows and check balances
flows = CF @ x  # cash received from bonds each date
surplus = np.zeros(n_dates)
surplus[0] = flows[0] - needs[0]
for t in range(1, n_dates):
    surplus[t] = grow * surplus[t-1] + flows[t] - needs[t]

summary_cf = pd.DataFrame({
    "Date": dates,
    "Bond Cash In ($mm)": np.round(flows, 6),
    "Required Outflow ($mm)": needs,
    "Surplus End ($mm)": np.round(surplus, 6)
})

print("=== Optimal Portfolio (Face Values in $mm) ===")
display(portfolio[["Maturity","Coupon","Price","FaceValue_mm","Cost_mm"]])

print(f"\nTotal Upfront Cost: ${total_cost:,.4f} mm")

print("\n=== Cash Flow & Surplus Check ===")
display(summary_cf)

# Sanity checks
assert np.all(surplus >= -1e-6), "Negative surplus detected."
print("\nAll constraints satisfied. ✅")

=== Optimal Portfolio (Face Values in $mm) ===


,Maturity,Coupon,Price,FaceValue_mm,Cost_mm
0,2024-01-01,0.0700,1.00,11.446482,11.446482
1,2024-07-01,0.0750,1.03,0.000000,0.000000
2,2025-01-01,0.0675,1.02,0.967352,0.986699
3,2025-07-01,0.0000,0.81,7.117342,5.765047
4,2026-01-01,0.1000,1.16,0.000000,0.000000
5,2026-07-01,0.0900,1.15,0.000000,0.000000
6,2027-01-01,0.1025,1.23,0.000000,0.000000
7,2027-07-01,0.1000,1.25,0.000000,0.000000



Total Upfront Cost: $18.1982 mm

=== Cash Flow & Surplus Check ===


,Date,Bond Cash In ($mm),Required Outflow ($mm),Surplus End ($mm)
0,2024-01-01,11.879757,7.5,4.379757
1,2024-07-01,0.032648,4.5,0.000000
2,2025-01-01,1.000000,1.0,0.000000
3,2025-07-01,7.117342,1.0,6.117342
4,2026-01-01,0.000000,1.0,5.239689
5,2026-07-01,0.000000,1.0,4.344483
6,2027-01-01,0.000000,1.0,3.431373
7,2027-07-01,0.000000,3.5,0.000000



All constraints satisfied. ✅
